lab7 - Разложение Холецкого, LU-разложение

Вспомогательные функции

In [36]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys

# согласованная матричная норма
def matrix_norm(matrix):
  sum = 0
  for i in range(len(matrix)):
    sum += abs(matrix[i])
  return max(sum)

def generate_symmetrical_matrix(l, r, n):
    a = np.random.uniform(l, r, (n, n))
    a = np.tril(a) + np.tril(a, -1).T
    return a

def increase_diag_elems(a, diag):
    n = len(a)
    for i in range(0, len(a)):
        a[i][i] = diag * sum(abs(a[i][j]) if j != i else 0 for j in range(n))
    return a

# Вычисляем диагональное преобладание
# Проверить выполнение условия диагонального преобладания
def calc_diagonal_dominance(a):
  degree = max(abs(a[i][i]) - sum(abs(a[i][j]) if j != i else 0 for j in range(len(a))) for i in range(len(a)))
  return degree > 0

In [40]:
def cholesky(A):
    # A = L L^T
    n = len(A)
    L = np.zeros_like(A)

    L[0][0] = np.sqrt(A[0][0])

    for i in range(1, n):
        for j in range(i):
            s = sum(L[i][p] * L[j][p] for p in range(j))
            L[i][j] = (A[i][j] - s) / L[j][j]
        L[i][i] = np.sqrt(A[i][i] - sum(L[i][p] ** 2 for p in range(i)))
    return np.array(L)

LU разложение

In [41]:
def LU(A):
    # A = L * U
    n = len(A)
    L, U = np.zeros_like(A), np.zeros_like(A)
    for i in range(n): L[i][i] = 1.0

    for i in range(n):
        for j in range(n):
            if i <= j:
                s = sum(L[i][k] * U[k][j] for k in range(i))
                U[i][j] = A[i][j] - s
            elif i > j:
                s = sum(L[i][k] * U[k][j] for k in range(j))
                L[i][j] = (A[i][j] - s) / U[j][j]
    return L, U

Тестирование. Сравнение погрешностей, используя матричные нормы

In [46]:
n = 3
A = generate_symmetrical_matrix(10, 40, n)
A = increase_diag_elems(A, 3)
print(f'matrix A:\n{A}\n')
# Holetski
L = cholesky(A)
LT = L.T
print(f'matrix L:\n{L}\n')
print(f'matrix L^T:\n{LT}\n')
print(f'matrix L * L^T:\n{L @ LT}\n')
holetsky_err = matrix_norm(A - L @ L.T)
print('------------------------------------------')
# LU
L, U = LU(A)
print(f'matrix L:\n{L}\n')
print(f'matrix U:\n{U}\n')
print(f'matrix L*U:\n{L @ U}\n')
lu_err = matrix_norm(A - L @ U)
print(f'Holetski decomposition error: {holetsky_err}\n')
print(f'LU decomposition error: {lu_err}\n')

matrix A:
[[147.88380495  21.91191064  27.38269101]
 [ 21.91191064 174.45481911  36.23969573]
 [ 27.38269101  36.23969573 190.86716023]]

matrix L:
[[12.16074854  0.          0.        ]
 [ 1.80185542 13.0846527   0.        ]
 [ 2.25172743  2.45955389 13.40699364]]

matrix L^T:
[[12.16074854  1.80185542  2.25172743]
 [ 0.         13.0846527   2.45955389]
 [ 0.          0.         13.40699364]]

matrix L * L^T:
[[147.88380495  21.91191064  27.38269101]
 [ 21.91191064 174.45481911  36.23969573]
 [ 27.38269101  36.23969573 190.86716023]]

------------------------------------------
matrix L:
[[1.         0.         0.        ]
 [0.14816978 1.         0.        ]
 [0.18516355 0.18797242 1.        ]]

matrix U:
[[147.88380495  21.91191064  27.38269101]
 [  0.         171.20813616  32.18240847]
 [  0.           0.         179.74747847]]

matrix L*U:
[[147.88380495  21.91191064  27.38269101]
 [ 21.91191064 174.45481911  36.23969573]
 [ 27.38269101  36.23969573 190.86716023]]

Holetski decompos